In [ ]:
# default_exp visualizations

In [2]:
#export
import sys
sys.path.append('/Users/constantin/workspace/EmpiRe/MS-EmpiRe_Python/')
from ms_empire.diffquant_utils import *

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np

def plot_pvals(result_df):
    pvals = result_df["peptide_pval"].to_list()
    plt.hist(pvals,99,cumulative=True,density=True, histtype='step')
    x = np.linspace(0,1,100)
    plt.plot(x, x)
    plt.show()

In [ ]:
#export
from scipy.stats import norm
import matplotlib.pyplot as plt

def plot_bgdist(bgdist):
    fc2counts_rescaled = tranform_fc2count_to_fc_space(bgdist.fc2counts, bgdist.cumulative[-1],1/100.0)
    
    plt.bar(list(fc2counts_rescaled.keys()), fc2counts_rescaled.values(),width=0.01,color='g')
    axes2 = plt.twinx()
    x = np.linspace(-4, 4, 1000)
    axes2.plot(x, norm.pdf(x, 0, bgdist.SD)/1.15)
    axes2.set_ylim(0.0, 0.4)
    plt.show()

def tranform_fc2count_to_fc_space(fc2counts, num_fcs, rescale_factor):
    fc2counts_fcscales = {}
    for fc, count in fc2counts.items():
        fc2counts_fcscales[fc*rescale_factor] = count/num_fcs

    return fc2counts_fcscales

In [ ]:
#export
import matplotlib.pyplot as plt
from scipy import stats

def plot_betweencond_fcs(df_c1_normed, df_c2_normed, get_median):
    if get_median:
        df_c1_normed = df_c1_normed.median(axis = 1, skipna = True).to_frame()
        df_c2_normed = df_c2_normed.median(axis = 1, skipna = True).to_frame()
    both_idx = df_c1_normed.index.intersection(df_c2_normed.index)
    df1 = df_c1_normed.loc[both_idx]
    df2 = df_c2_normed.loc[both_idx]

    for col1 in df1.columns:
        for col2 in df2.columns:
            diff_fcs = df1[col1].to_numpy() - df2[col2].to_numpy()
            median = np.nanmedian(diff_fcs)
            #plt.axvline(mode, color = 'blue')
            plt.axvline(median, color = 'red')
            plt.hist(diff_fcs,99,density=True, histtype='step', range=(-3.5,3.5))
    plt.xlabel("log2(fc)")
    plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
from scipy import stats
import itertools

def plot_withincond_fcs(df, xlim = None):
    combs = list(itertools.combinations(df.columns, 2))
    for cpair in combs:
        col1 = cpair[0]
        col2 = cpair[1]
        diff_fcs = df[col1].to_numpy() - df[col2].to_numpy()
        mode = stats.mode(diff_fcs, nan_policy='omit')[0][0]
        median = np.nanmedian(diff_fcs)
        #plt.axvline(mode, color = 'blue')
        #plt.axvline(median, color = 'red')
        plt.hist(diff_fcs,99,density=True, histtype='step',range=(-2,2))
        plt.xlabel("log2 peptide fcs")
        
    if xlim is not None:
        plt.xlim(xlim) 
    plt.show()

In [ ]:
from scipy import stats
mode = stats.mode([1,3,4,5,6,7,8,4,3,2,46,1,1,1])[0]
print(mode)

[1]


In [ ]:
#export
import matplotlib.pyplot as plt
def scatter_df_columns(merged_df, log_axes = False):
    col = (0.2, 0.4, 0.6, 0.1)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        ax_p = merged_df.plot.scatter(x=ref,y=compare, color = col)
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        x = np.linspace(0,merged_df[ref].max(),100)
        plt.plot(x, x)
        if log_axes:
            plt.xscale('log')
            plt.yscale('log')
        plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
def plot_cumhist_dfcols(merged_df):
    col = (0.2, 0.4, 0.6, 0.4)
    ref_columns = list(filter(lambda x : "_ref" in x, merged_df.columns.to_list())) #filter the reference columns from the merged df

    for ref in ref_columns:
        compare = ref.replace("_ref", "")
        plt.hist(merged_df[ref], 100, density=True, histtype='step', label='reference')
        plt.hist(merged_df[compare], 100, density=True, histtype='step',label='compare')
        corr = merged_df[ref].corr(merged_df[compare])
        plt.title(f"{ref} vs. {compare} corr {corr}")
        plt.show()

In [ ]:
#export
from matplotlib_venn import venn2
from matplotlib import pyplot as plt
def compare_peptid_protein_overlaps(protein_ref, protein_comp, peptide_ref, peptide_comp):
    protIDs_ref = set(protein_ref["protein"].to_list())
    protIDs_comp = set(protein_comp["protein"].to_list())
    venn2([protIDs_ref, protIDs_comp], ('protIDs_ref', 'protIDs_comp'))
    plt.show()
    pepIDs_ref = set(peptide_ref["ion"].to_list())
    pepIDs_comp = set(peptide_comp["ion"].to_list())
    venn2([pepIDs_ref, pepIDs_comp], ('pepIDs_ref', 'pepIDs_comp'))
    plt.show()

In [ ]:
#export
import matplotlib.pyplot as plt
import seaborn as sns

def plot_fold_change(df, key1, key2):
    to_plot = df.copy()
    to_plot[f'Ratio ({key1}/{key2})'] = np.log2(to_plot[key1] / to_plot[key2])
    to_plot[f'Inten_{key1}'] = np.log10(to_plot[key1])

    species = 'Human'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'Species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')
    species='Ecoli'
    val = to_plot.loc[to_plot['species']==species, f'Ratio ({key1}/{key2})'].values
    val = val[~np.isnan(val)&~np.isinf(val)&~np.isneginf(val)]
    print(f'species={species}, n={len(val)}, median={np.median(val)}, dev={np.std(val)}')

    plt.figure(figsize=(7,7))
    ax = sns.scatterplot(x=f'Ratio ({key1}/{key2})', y=f'Inten_{key1}', hue="species", data=to_plot, alpha=0.5)
    plt.title('Fold Change')
    plt.xlim([-4.5, 6.5])
    #plt.ylim([6,11.5])
    plt.show()


In [ ]:
#export
import matplotlib.pyplot as plt

import numpy as np

def volcano_plot(result_df, fc_header = "log2fc", fdr_header = "fdr", significance_cutoff = 0.05, log2fc_cutoff = 0.5,ybound = None, xbound =None):
    result_df[fdr_header] = result_df[fdr_header].replace(0, np.min(result_df[fdr_header].replace(0, 1.0)))
    fdrs = result_df[fdr_header].to_numpy()
    fcs = result_df[fc_header].to_numpy()
    sighits_down = sum((fdrs<significance_cutoff) & (fcs <= -log2fc_cutoff))
    sighits_up = sum((fdrs<significance_cutoff) & (fcs >= log2fc_cutoff))
    plt.title(f"{sighits_up} up, {sighits_down} down of {len(fcs)}")
    plt.scatter(result_df[fc_header],-np.log10(result_df[fdr_header]),s=10, c='grey', alpha = 0.1)
    plt.xlabel('log2 FC',fontsize = 14)
    plt.ylabel('-log10 FDR',fontsize = 14)

    if ybound==None:
        plt.ylim(0,max(-np.log10(result_df[fdr_header]))+0.5)
    else:
        plt.ylim(ybound)
    if significance_cutoff>0:
        plt.axhline(y=-np.log10(significance_cutoff), color='g', linestyle='-')
    if log2fc_cutoff >0:
        plt.axvline(x=log2fc_cutoff, color='g', linestyle='-')
        plt.axvline(x=-log2fc_cutoff, color='g', linestyle='-')
    maxfc = max(abs(result_df[fc_header]))+0.5
    if xbound==None:
        plt.xlim(-maxfc,maxfc)
    else:
        plt.xlim(xbound)
    plt.show()


In [3]:
#export
import seaborn as sns
import matplotlib.pyplot as plt

def beeswarm_ion_plot(protein,diffresults_df,normed_df, sample2cond,saveloc = None,ion_header = 'ion'):
    diffresults_line = diffresults_df.loc[protein]
    fdr = diffresults_line["fdr"]
    value_vars = set.intersection(set(normed_df.columns), set(sample2cond.keys()))
    protein_df = normed_df.loc[[protein]]
    df_melted = pd.melt(protein_df, value_vars= value_vars, id_vars=[ion_header], value_name="intensity", var_name="sample")
    df_melted["condition"] = [sample2cond.get(x) for x in df_melted["sample"]]
    pal2 = [(0.94, 0.94, 0.94),(1.0, 1.0, 1.0)]
    ax = sns.boxplot(x="ion", y="intensity", hue="condition", data=df_melted, palette=pal2)
    ax = sns.swarmplot(x="ion", y="intensity", hue="condition", data=df_melted, palette="Set2", dodge=True)
    handles, labels = ax.get_legend_handles_labels()

    l = plt.legend(handles[2:4], labels[2:4])

    plt.xticks(rotation=90)
    if "gene" in diffresults_df.columns:
        gene = diffresults_line["gene"].values[0]
        plt.title(f"{gene} ({protein}) FDR: {fdr:e.1}")
    else:
        plt.title(f"{protein} FDR: {fdr:e}")
    if saveloc is not None:
        plt.savefig(saveloc)
    
    plt.show()

In [ ]:
#export
import pandas as pd

def initialize_result_dataframes_per_condpair(cond1, cond2, result_folder, diffresults = "default", samplemap = "samples.map"):
    'reads the standard AQ output tables for a given pair of conditions and initializes diffprots_df, normed_peptides_df, and sample2cond_dict'
    condpair = get_condpairname([cond1, cond2])
    if diffresults == "default":
        diffresults = f"{result_folder}/diffresults/{condpair}.results.tsv" 
    diffprots = pd.read_csv(diffresults, sep = "\t")
    diffprots = diffprots[(diffprots["condpair"] == condpair)]
    samplemap_df, sample2cond = initialize_sample2cond(samplemap)
    normed_peptides = pd.read_csv(f"{result_folder}/diffresults/{condpair}.normed.tsv", sep = "\t")
    normed_peptides[samplemap_df["sample"].values] = np.log2(normed_peptides[samplemap_df["sample"].values].replace(0, np.nan))
    diffprots["-log10fdr"] = -np.log10(diffprots["fdr"])
    diffprots = diffprots.set_index("protein")
    normed_peptides = normed_peptides.set_index("protein")
    return diffprots, normed_peptides, sample2cond


In [ ]:
#export
import pandas as pd

def initialize_sample2cond(samplemap = "samples.map"):
    samplemap_df = pd.read_csv(samplemap, sep = "\t")
    sample2cond = dict(zip(samplemap_df["sample"], samplemap_df["condition"]))
    return samplemap_df, sample2cond